## AI Agent智能应用从0到1定制开发 
## AI Agent Intelligent Application Custom Development from 0 to 1
******
- 此代码为网课《AI Agent智能应用从0到1定制开发》的配套代码，需要注意本套代码建议与网课适配配合食用。
- This code for the online course <AI Agent Intelligent Applications from 0 to 1 custom development> supporting code, need to pay attention to this set of code is recommended with the online course adapted to work with consumption.
- 需要注意由于课程开发周期的原因，langchain版本跨越了3个大版本，部分代码会与视频演示有差别!
- Note that due to the course development cycle, the langchain version spans 3 major releases and some of the code will differ from the video demo!
- 课程地址：https://coding.imooc.com/class/822.html
- Course address: https://coding.imooc.com/class/822.html

### 从环境变量中读取密钥
### Read the key from the environment variable
- 注意：尽量将你的OpenAI Key存储在类似.env文件中，而不是明文暴露在代码里，这是一种基本的安全措施
- Note: Try to store your OpenAI Key in something like an .env file, rather than exposing it explicitly in code, as a basic safety measure!
******

In [12]:

import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("asset/openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

### 如何给agent正确的增加记忆
### How to add memory to agent correctly 
- 将memory插入到提示词模板中
- Inserting memory into the prompt word template
<hr>

In [13]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder


- 定义大模型
- defind LLM

In [14]:

llm=ChatOpenAI(
    temperature=0,
    model="gpt-4-1106-preview",
    )

- 构建agent可用工具
- Build agent available tools

In [15]:
#构建一个搜索工具
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain.from_llm(
    llm=llm,
    verbose=True
)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

[Tool(name='Search', description='useful for when you need to answer questions about current events or the current state of the world', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='e8c972251bd371ef03c23b9bc9e1ad49cb8cb7823befbfda0c8980d7a66b5fa0', aiosession=None)>), Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n

- 增加memory组件
- add memory component

In [16]:
#记忆组件
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

- 定义agent
- defind agent

In [17]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
    )

In [18]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
prompt=PromptTemplate(input_variables=['input'], template='{input}')
variable_name='agent_scratchpad'


In [19]:
agent_chain.invoke("我叫什么名字？")



> Entering new AgentExecutor chain...


作为一个AI，我无法知道您的具体信息，除非您告诉我。所以目前我不知道您的名字。您愿意告诉我您的名字吗？

> Finished chain.


{'input': '我叫什么名字？',
 'chat_history': [HumanMessage(content='我叫什么名字？'),
  AIMessage(content='作为一个AI，我无法知道您的具体信息，除非您告诉我。所以目前我不知道您的名字。您愿意告诉我您的名字吗？')],
 'output': '作为一个AI，我无法知道您的具体信息，除非您告诉我。所以目前我不知道您的名字。您愿意告诉我您的名字吗？'}

- 需要使用agent_kwargs传递参数，将chat_history传入
-  Need to pass parameters using agent_kwargs, pass chat_history into the

In [20]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory #记忆组件
    )

- 查看重新渲染后的提示词模版
- View the re-rendered cue word template

In [21]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

[SystemMessage(content='You are a helpful AI assistant.'), MessagesPlaceholder(variable_name='chat_history'), MessagesPlaceholder(variable_name='agent_scratchpad'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]
content='You are a helpful AI assistant.'
variable_name='chat_history'
variable_name='agent_scratchpad'


In [22]:
agent_chain.invoke("好厉害，刚才我们都聊了什么？")



> Entering new AgentExecutor chain...
作为AI，我可以回顾我们的对话历史。到目前为止，我们的对话内容如下：

1. 您问我：“我叫什么名字？”
2. 我回答说，作为AI，我无法知道您的名字，除非您告诉我。
3. 您评论说：“好厉害”，然后问我我们都聊了什么。

这就是我们目前为止的对话内容。如果您有其他问题或需要帮助，请随时告诉我！

> Finished chain.


{'input': '好厉害，刚才我们都聊了什么？',
 'chat_history': [HumanMessage(content='我叫什么名字？'),
  AIMessage(content='作为一个AI，我无法知道您的具体信息，除非您告诉我。所以目前我不知道您的名字。您愿意告诉我您的名字吗？'),
  HumanMessage(content='好厉害，刚才我们都聊了什么？'),
  AIMessage(content='作为AI，我可以回顾我们的对话历史。到目前为止，我们的对话内容如下：\n\n1. 您问我：“我叫什么名字？”\n2. 我回答说，作为AI，我无法知道您的名字，除非您告诉我。\n3. 您评论说：“好厉害”，然后问我我们都聊了什么。\n\n这就是我们目前为止的对话内容。如果您有其他问题或需要帮助，请随时告诉我！')],
 'output': '作为AI，我可以回顾我们的对话历史。到目前为止，我们的对话内容如下：\n\n1. 您问我：“我叫什么名字？”\n2. 我回答说，作为AI，我无法知道您的名字，除非您告诉我。\n3. 您评论说：“好厉害”，然后问我我们都聊了什么。\n\n这就是我们目前为止的对话内容。如果您有其他问题或需要帮助，请随时告诉我！'}